## Analysis of Algorithmic Merge (WIP)
Compared against hand-crafted data set.

In [1]:
from importlib import reload

import pandas as pd
import gensim.downloader as api

import merge

In [2]:
pd.set_option('display.max_rows', 1000)

In [3]:
%time word_vectors = api.load("glove-wiki-gigaword-300") # 50, 100, 200, 300 sizes available

CPU times: total: 1min 11s
Wall time: 1min 11s


In [8]:
%%time
DATA_PATH = '../Data/'
HAND_FILE = 'Copy of Max, Samantha, Maria data with transcipt data.xlsx'
DTYPES = {'ID': 'uint8', 'Session': 'uint8', 'Receipt': 'uint8', 'Item': 'string'}


df_algo = pd.read_csv(DATA_PATH + 'merged_full.csv', index_col=0, usecols=[0, 1, 2, 3, 4], dtype=DTYPES)
df_hand = pd.read_excel(DATA_PATH + HAND_FILE, sheet_name='Data everyone', usecols=[0, 1, 2, 4])

CPU times: total: 969 ms
Wall time: 998 ms


C:\Users\josep\Documents\Projects\ALab\CookiesAndCognition\DataCleaning\envs\lib\site-packages\openpyxl\worksheet\_reader.py:211: UserWarning: Cell D3456 is marked as a date but the serial value 6684137 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [5]:
df_algo.head()

,ID,Session,Receipt,Item
0,129,1,1,grapefruit soda
1,129,1,1,grapefruit soda
2,129,1,1,dried fruit
3,129,1,1,lemon wafers
4,129,1,1,vanilla wafers


In [6]:
df_algo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4085 entries, 0 to 4084
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       4085 non-null   uint8 
 1   Session  4085 non-null   uint8 
 2   Receipt  4085 non-null   uint8 
 3   Item     4085 non-null   string
dtypes: string(1), uint8(3)
memory usage: 75.8 KB


In [9]:
df_hand.head()

,ID,TP-Date,RecNum,Item (modifier)
0,131,1.0,1.0,Strawberries
1,131,1.0,1.0,Apple Cider Vinegar
2,131,1.0,1.0,Cheddar Cheese (Sticks)
3,131,1.0,2.0,Candy Bar (Baby Ruth)
4,131,1.0,2.0,Flavored Water


In [10]:
df_hand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4433 entries, 0 to 4432
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               4326 non-null   object 
 1   TP-Date          4305 non-null   float64
 2   RecNum           2344 non-null   float64
 3   Item (modifier)  4272 non-null   object 
dtypes: float64(2), object(2)
memory usage: 138.7+ KB


In [11]:
df_hand.columns = ['ID', 'Session', 'Receipt', 'Item']

In [12]:
df_hand.ID.unique()

array([131, nan, '131 Hit rate', 139, '139 hit rate ', 145,
       '145 hit rate', 149, '149 hit rate', 152, '152 Hit rate ', 157,
       162, 113, '113 hit rate ', 118, '118 hit rate ', 126,
       '126 hit rate ', 115, '115 hit rate', 110, '110 hit rate', 154,
       '154 hit rate ', 150, '150 hit rate ', 143, '143 hit rate', 146,
       '146 Hit rate', 148, '148 hit rate', 128, '128 hit rate ', 119,
       '119 hit rate', 134, '134 hit rate', 129, '129 hit rate', 136,
       '136 hit rate ', 144, '144 hit rate'], dtype=object)

In [13]:
df_hand = df_hand[df_hand.ID.astype(str).str.isdigit()]

In [14]:
df_hand.ID.unique()

array([131, 139, 145, 149, 152, 157, 162, 113, 118, 126, 115, 110, 154,
       150, 143, 146, 148, 128, 119, 134, 129, 136, 144], dtype=object)

In [15]:
df_algo.ID.unique()

array([129, 136, 144, 154, 159, 128, 119, 134, 131, 139, 145, 149, 157,
       162, 113, 118, 126, 114, 137, 153, 127, 130, 135, 148], dtype=uint8)

In [16]:
set(df_algo.ID.unique()) - set(df_hand.ID.unique())

{114, 127, 130, 135, 137, 153, 159}

In [17]:
set(df_hand.ID.unique()) - set(df_algo.ID.unique())

{110, 115, 143, 146, 150, 152}

In [18]:
df_hand = df_hand.dropna()

In [19]:
set(df_hand.ID.unique()) - set(df_algo.ID.unique())

set()

In [20]:
ids_assigned_all = {121, 114, 137, 153, 141, 127, 130, 135, 148, 158}
set(df_hand.ID.unique()) & (ids_assigned_all)

{148}

In [21]:
shared_ids = set.intersection(set(df_hand.ID.unique()), set(df_algo.ID.unique()))
print(shared_ids)

{128, 129, 162, 131, 134, 136, 139, 144, 145, 113, 148, 149, 118, 119, 154, 157, 126}


In [22]:
df_hand = df_hand[df_hand.ID.isin(shared_ids)]
df_algo = df_algo[df_algo.ID.isin(shared_ids)]

In [23]:
print(df_hand.info())
print(df_algo.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2320 entries, 0 to 4431
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       2320 non-null   object 
 1   Session  2320 non-null   float64
 2   Receipt  2320 non-null   float64
 3   Item     2320 non-null   object 
dtypes: float64(2), object(2)
memory usage: 90.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2867 entries, 0 to 4084
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       2867 non-null   uint8 
 1   Session  2867 non-null   uint8 
 2   Receipt  2867 non-null   uint8 
 3   Item     2867 non-null   string
dtypes: string(1), uint8(3)
memory usage: 53.2 KB
None


Where do these 500 additional data points come from? Possibly the Quantity data?

In [24]:
df_hand.Item.value_counts()

Bananas                       35
strawberries                  27
raspberries                   20
Blueberries                   17
blueberries                   17
                              ..
gloves                         1
nitrile gloves                 1
lysol cleaner                  1
clear care solution            1
Dog Treats (Peanut Butter)     1
Name: Item, Length: 1293, dtype: int64

In [25]:
df_hand[df_hand.Item == "Bananas"]

,ID,Session,Receipt,Item
38,131,2.0,4.0,Bananas
134,139,1.0,2.0,Bananas
401,145,1.0,2.0,Bananas
439,145,2.0,1.0,Bananas
499,145,3.0,3.0,Bananas
552,145,4.0,1.0,Bananas
567,145,4.0,2.0,Bananas
596,145,4.0,2.0,Bananas
628,149,1.0,2.0,Bananas
646,149,1.0,4.0,Bananas


In [26]:
df_hand[df_hand.duplicated(keep=False)]

,ID,Session,Receipt,Item
20,131,2.0,2.0,Cookies
22,131,2.0,2.0,Cookies
132,139,1.0,2.0,Carrots
135,139,1.0,2.0,Peaches
146,139,1.0,2.0,Carrots
154,139,1.0,2.0,Peaches
536,145,4.0,1.0,Cucumbers
543,145,4.0,1.0,Peppers (tabasco)
550,145,4.0,1.0,Cucumbers
558,145,4.0,1.0,Peppers (tabasco)


In [27]:
df_algo[df_algo.duplicated(keep=False)]

,ID,Session,Receipt,Item
0,129,1,1,grapefruit soda
1,129,1,1,grapefruit soda
9,129,1,1,eggplant spread
10,129,1,1,eggplant spread
15,129,1,1,sweet kale salad
...,...,...,...,...
4074,148,2,2,mask
4075,148,2,2,mask
4082,148,2,2,sprays
4083,148,2,2,sprays


### Examination of Variation within Sessions

In [28]:
pd.concat([df.groupby(by=['ID', 'Session']).Item.count() for df in [df_algo, df_hand]], axis=1, ignore_index=True)

0    1
ID  Session          
113 1         27   21
    2         33   28
    4         33   28
    5         31   26
    6         36   29
118 1        115   87
119 1         43   29
    2         28   25
    3         59   43
    5        127   88
    6         60   37
126 1         50   26
128 1         91   78
    2         62   53
    3         48   39
    4         47   46
    5         94   73
129 1         17   15
    2         25   29
    3         84   74
    4         47   58
    5         21   23
131 1         37   17
    2         29   21
    3         20   14
    4         10   10
    5         14   23
134 3         78   60
136 1         45   59
    2         53   56
    4         55   65
    5         76  103
139 1         88   80
144 1         86  137
145 1         63   48
    2         78   54
    3         47   38
    4         67   61
148 2         65   47
149 1         59   39
    2         27   17
    3         65   35
    4         43   24
    5         32   14
    6         41   24
154 1         28   28
    2         91   54
    3        115   55
    4         77   51
    5        135   80
157 1         44   38
162 1         21   13